# PROMETEO_v1.0

## Paquetería

In [1]:
import ast
import numpy as np
import os
import pandas as pd
import pyperclip
import textwrap
import tiktoken
import umap.umap_ as umap
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from typing import Optional

## Funciones

In [2]:
def num_tokens_from_string(string: str) -> int:
    """Cuenta el número de tokens en el documento
    proporcionado."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """Esta función toma un conjunto de embeddings de
    alta dimensión y los reduce a una dimensión menor (2)."""
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    """Obtiene el número óptimo de clústers."""
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """Clusteriza con el método GMM."""
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters

def format_cluster_texts(df):
    """Agrupa los textos de cada clúster en listas."""
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Texto'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts

def wrap_text_preserve_newlines(text, width=80):
    """Formato para respuestas."""
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    """Generador de referencias."""
    print(wrap_text_preserve_newlines(llm_response['answer']))
    print('\nReferencias:')
    for contexto in llm_response["context"][:5]:
        print(contexto)
    print('\n\n')

## Parámetros

In [3]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings(
)

detailed_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini'
)

template = """Tu tarea es generar un resumen extremadamente detallado del siguiente
texto: {text} """

prompt = PromptTemplate.from_template(template)
chain = prompt | detailed_llm | StrOutputParser()

llm = ChatOpenAI(
    temperature=0.5,
    model_name='gpt-4o-mini'
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

## ¿Quieres crear o cargar un tema de conversación?

In [9]:
user_input = input("¿Crear (1) o cargar (2) un tema de conversación?: ")

if user_input.lower() == "1":
    print(f'Elegiste crear de cero un tema de conversación.\n')

    # Asegúrate de que haya PDFs en la carpeta 'docs'
    documents = DirectoryLoader('./docs/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
    # Tratameinto de caracteres indeseados
    for d in documents:
        d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')

    docs = text_splitter.split_documents(documents)
    texts = [doc.page_content for doc in docs]

    d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
    d_reversed = list(reversed(d_sorted))
    concatenated_content = "\n\n\n --- \n\n\n".join(
        [doc.page_content for doc in d_reversed]
    )
    print(
        "Número de tokens en el documento proporcionado: %s"
        % num_tokens_from_string(concatenated_content)
    )

    global_embeddings = [embeddings.embed_query(txt) for txt in texts]

    topic_name = input('¿Cómo se llama el tema de conversación?')

    embed_name = topic_name + '_emb' + '.txt'
    with open(rf'./{embed_name}', 'w') as f:
        for i in global_embeddings:
            f.write("%s\n" % i)
    print(f'\nEstás usando el tema de conversación: {embed_name}')

    dim = 2
    global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)
    labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)
    simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

    df = pd.DataFrame({
        'Texto': texts,
        'Embedding': list(global_embeddings_reduced),
        'Cluster': simple_labels
    })

    clustered_texts = format_cluster_texts(df)

    print(f"Número de clústers {len(clustered_texts)}")

    summaries = {}
    for cluster, text in clustered_texts.items():
        summary = chain.invoke({"text": text})
        summaries[cluster] = summary
    embedded_summaries = [embeddings.embed_query(summary) for summary in summaries.values()]
    embedded_summaries_np = np.array(embedded_summaries)
    labels, _ = gmm_clustering(embedded_summaries_np, threshold=0.5)
    simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]
    clustered_summaries = {}
    for i, label in enumerate(simple_labels):
        if label not in clustered_summaries:
            clustered_summaries[label] = []
        clustered_summaries[label].append(list(summaries.values())[i])
    final_summaries = {}
    for cluster, texts in clustered_summaries.items():
        combined_text = ' '.join(texts)
        summary = chain.invoke({"text": combined_text})
        final_summaries[cluster] = summary

    texts_from_df = df['Texto'].tolist()
    texts_from_clustered_texts = list(clustered_texts.values())
    texts_from_final_summaries = list(final_summaries.values())

    combined_texts = texts_from_df + texts_from_clustered_texts + texts_from_final_summaries

    file_name = topic_name + '.txt'
    with open(file_name, 'w', encoding='utf-8') as f:
        for t in combined_texts:
            f.write("%s\n" % t)

    with open(file_name, 'r', encoding='utf-8') as f:
        content = f.read()

    textos = text_splitter.split_text(content) #chunked_knowledge in v2

    persist_directory = topic_name + '_kb'
    vectorstore = Chroma.from_texts(texts=textos,
                                    embedding=embeddings,
                                    persist_directory=persist_directory)
    vectorstore.persist()
    vectorstore = None

    vectorstore = Chroma(persist_directory=persist_directory,
                         embedding_function=embeddings)

    def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
        final_number = initial_number
        while final_number < max_threshold:
            retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
            docs = retriever.invoke(string)
            text = "".join([doc.page_content for doc in docs])
            if num_tokens_from_string(text) < max_threshold:
                final_number += 1
            else:
                break
        return final_number

    final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
    print(f'\nK final es: {final_number}')
    retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
    
elif user_input.lower() == "2":
    print('Elegiste cargar un tema de conversación ya creado.\n')
    # global_embeddings = []

    topic_name = input('¿Cómo se llama el tema de conversación?')

    embed_name = topic_name + '_emb' + '.txt'
    print(f'Estás usando el tema de conversación: {embed_name}\n')
    #with open(rf'./{embed_name}', 'r') as f:
    #    for i in f:
    #        x = ast.literal_eval(i.strip())  # Convertir la cadena a lista de números
    #        global_embeddings.append(x)

    # global_embeddings = np.array(global_embeddings, dtype=float)

    #file_name = topic_name + '.txt'
    #with open(file_name, 'r', encoding='utf-8') as f:
    #    content = f.read()

    persist_directory = topic_name + '_kb'
    vectorstore = Chroma(persist_directory=persist_directory, 
                    embedding_function=embeddings)

    def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
        final_number = initial_number
        while final_number < max_threshold:
            retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
            docs = retriever.invoke(string)
            text = "".join([doc.page_content for doc in docs])
            if num_tokens_from_string(text) < max_threshold:
                final_number += 1
            else:
                break
        return final_number

    final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
    print(f'K final es: {final_number}')
    retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
    
elif user_input != "1" and user_input != "2":
    print('No seleccionaste ningún tema de conversación.\n')

# Se personaliza el LLM #

template = """
Eres Prometeo, un asistente especializado en revisión bibliográfica, que habla Español.

Tu tarea consiste en proporcionar respuestas extremadamente detalladas y basadas en evidencia a 
cualquier pregunta relacionada con el siguiente contexto, obtenido de un artículo científico: {context}.

Tu respuesta debe centrarse en los aspectos más relevantes de la literatura científica sin mencionar 
directamente el contexto proporcionado. Identifica y utiliza palabras clave para enfocarte en los temas 
más importantes para dar una respuesta más precisa.

Siempre responde en Español, excepto en nombres propios, y no menciones detalles sobre ti a menos que se 
te pregunte directamente.

Finalmente y teniendo en cuento lo anterior, responde la siguiente pregunta: {question}
"""

selected_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) #WTF is this doing?

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | selected_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain
)

Elegiste cargar un tema de conversación ya creado.

Estás usando el tema de conversación: pppp_4_emb.txt

K final es: 29


In [ ]:
# script para múltiples preguntas a la vez

preguntas_lab = [
'¿Cuál es el título del documento?',
'¿Puedes generar una cita en formato APA del documento?',
'¿Cuál es el tema principal del documento?',
'Según el autor, ¿cómo se define el aprendizaje experiencial?',
'Según el autor, ¿cuál o cuáles son las teorías que explican el aprendizaje experiencial',
'Según el autor, ¿cómo se puede aplicar el aprendizaje experiencial en contextos educativos?',
'Según el autor, ¿Cómo el aprendizaje experiencial contribuye a los objetivos de un laboratorio de investigación en ciencias empresariales?'
]

for p in preguntas_lab:
    query = p
    print(query + '\n')
    llm_response = rag_chain_with_source.invoke(query)
    process_llm_response(llm_response)
    print('\n\n')

In [8]:
### ACT LAB single question ###

query = input("Hazme una pregunta: ")
print(query + '\n')
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

# Según el autor, ¿qué actividades pueden desarrollarse en un laboratorio de investigación?

¿Quién es el autor del documento?

El documento es escrito por Pierre Desrochers y Frederic Sautet. Ambos autores
analizan el debate sobre las políticas de desarrollo económico regional,
centrándose en la especialización industrial versus la diversidad industrial. Su
trabajo explora la importancia del contexto local, la movilidad laboral, el
conocimiento tácito y la crítica a las políticas de especialización regional,
promoviendo en su lugar un enfoque que fomente la actividad emprendedora y la
innovación a través de entornos diversificados.

Referencias:
page_content='### 10. **Referencias Históricas y Ejemplos**
   - Se documentan ejemplos históricos de industrias localizadas en Inglaterra y se menciona la ley de la ventaja comparativa de David Ricardo.
   - Se discuten los intentos de fomentar "distritos industriales" y se critica la limitación de estas políticas a un análisis de insumo-producto.

### 11. **Implicaciones para la Política de Desarrollo**
   - Se sugiere que las polít

In [ ]:
###############
## UNWRAPER ###
###############

wrapped_text = input('Ingresa el texto a desenvolver: ')
unwrapped_text = textwrap.fill(wrapped_text, width=10000)
pyperclip.copy(unwrapped_text)
print('Texto desenvuelto copiado en el portapapeles.\n')
print(unwrapped_text)

# Hacer historial y guardar conversaciones para medir efectividad.